In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
from sqlalchemy import create_engine
import pandas as pd


In [19]:
with create_engine('sqlite:///results.db').connect() as conn: 
    table = pd.read_sql_table('result', conn)


In [20]:
table

,id,piece,correct_exp,correct_sample,selected_exp,selected_sample
0,28,kv331_1_start_-45_nodes_134_end_0,0e3ee2b957cff8903c6c1eb79457a6add2cf5c2c8e88c9...,0,0e3ee2b957cff8903c6c1eb79457a6add2cf5c2c8e88c9...,0
1,29,kv331_1_start_-45_nodes_134_end_0,0e3ee2b957cff8903c6c1eb79457a6add2cf5c2c8e88c9...,0,0e3ee2b957cff8903c6c1eb79457a6add2cf5c2c8e88c9...,0
2,30,kv331_1_start_-45_nodes_134_end_0,3bceac27346acddd96b11d908f3ac92fd667561f0f2fad...,0,3bceac27346acddd96b11d908f3ac92fd667561f0f2fad...,0
3,31,kv331_1_start_-45_nodes_134_end_0,80f8da596bf413a11494bedba1c1793778e648bda3bc2d...,0,80f8da596bf413a11494bedba1c1793778e648bda3bc2d...,0
4,32,kv331_1_start_-45_nodes_134_end_0,3bceac27346acddd96b11d908f3ac92fd667561f0f2fad...,0,3bceac27346acddd96b11d908f3ac92fd667561f0f2fad...,0
5,33,kv331_1_start_-45_nodes_134_end_0,3bceac27346acddd96b11d908f3ac92fd667561f0f2fad...,0,3bceac27346acddd96b11d908f3ac92fd667561f0f2fad...,0
6,34,kv331_1_start_-45_nodes_134_end_0,3bceac27346acddd96b11d908f3ac92fd667561f0f2fad...,0,3bceac27346acddd96b11d908f3ac92fd667561f0f2fad...,0
7,35,kv331_1_start_-45_nodes_134_end_0,0e3ee2b957cff8903c6c1eb79457a6add2cf5c2c8e88c9...,0,80f8da596bf413a11494bedba1c1793778e648bda3bc2d...,0
8,36,kv331_1_start_-45_nodes_134_end_0,3bceac27346acddd96b11d908f3ac92fd667561f0f2fad...,0,3bceac27346acddd96b11d908f3ac92fd667561f0f2fad...,0
9,37,kv331_1_start_-45_nodes_134_end_0,3bceac27346acddd96b11d908f3ac92fd667561f0f2fad...,0,80f8da596bf413a11494bedba1c1793778e648bda3bc2d...,0


In [21]:
corrects = (table.correct_exp == table.selected_exp)
corrects.mean()

0.8888888888888888